In [1]:
# I am using this file to extract the required pages based on my DOB
from google.colab import files
uploaded = files.upload()

Saving harrypotter.pdf to harrypotter.pdf


In [2]:
#Installing required packages
# PyPDF2 :to read the PDF
# mrjob :to implement MapReduce
# pyspellchecker :to detect non-English words
!pip install PyPDF2 mrjob pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 83.1 MB/s eta 0:00:00


In [3]:
# Reading the PDF and extracting required pages

# DOB: 19-05-2003

from PyPDF2 import PdfReader

pdf = PdfReader("harrypotter.pdf")

# extracting a range of pages and save as a text file
def extract_pages(start, end, output_file):
    text = ""

    for i in range(start - 1, end):   # -1 because page index starts from 0
        text += pdf.pages[i].extract_text()

    # Saving extracted text into a file
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(text)

# Birth date = 19 → file1 → pages 19 to 28
extract_pages(19, 28, "file1.txt")

# Birth year = 2003 → use 103 → file2 → pages 103 to 112
extract_pages(103, 112, "file2.txt")

print("Text files created successfully")

Text files created successfully


In [10]:
# Display first few characters to verify extraction

print(open("file1.txt").read(500))

“My	dear	Professor,	I’ve	never	seen	a	cat	sit	so	stiffly.”
“You’d	be	stiff	if	you’d	been	sitting	on	a	brick	wall	all	day,”	said	Professor
McGonagall.
“All	day?	When	you	could	have	been	celebrating?	I	must	have	passed	a
dozen	feasts	and	parties	on	my	way	here.”
Professor	McGonagall	sniffed	angrily.
“Oh	yes,	everyone’s	celebrating,	all	right,”	she	said	impatiently.	“You’d
think	they’d	be	a	bit	more	careful,	but	no	—	even	the	Muggles	have	noticed
something’s	going	on.	It	was	on	their	news.”	She	jer


In [15]:
%%file wordcount_mrjob.py

from mrjob.job import MRJob
import re

class MRWordCount(MRJob):

    def mapper(self, _, line):
        # extract only real words (remove punctuation & special symbols)
        words = re.findall(r"[a-zA-Z]+", line.lower())

        for word in words:
            yield word, 1

    def reducer(self, key, values):
        yield key, sum(values)

if __name__ == "__main__":
    MRWordCount.run()

Overwriting wordcount_mrjob.py


In [26]:
!python wordcount_mrjob.py file1.txt > wordcount_output.txt


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/wordcount_mrjob.root.20260226.054610.273899
Running step 1 of 1...
job output is in /tmp/wordcount_mrjob.root.20260226.054610.273899/output
Streaming final output from /tmp/wordcount_mrjob.root.20260226.054610.273899/output...
Removing temp directory /tmp/wordcount_mrjob.root.20260226.054610.273899...


In [27]:
# Display the COMPLETE output

with open("wordcount_output.txt") as f:
    print(f.read())

"faintly"	1
"fair"	1
"faltered"	1
"family"	1
"famous"	4
"fast"	3
"fat"	2
"fateful"	1
"father"	2
"favorite"	1
"feasts"	1
"feeling"	1
"feet"	2
"fell"	3
"few"	2
"finally"	2
"find"	3
"fine"	1
"firmly"	1
"first"	4
"five"	1
"fixed"	1
"flatter"	1
"flinched"	1
"flocks"	1
"flyin"	1
"flying"	2
"fond"	1
"for"	15
"forehead"	2
"forever"	1
"forgotten"	1
"forward"	1
"found"	3
"four"	4
"frightened"	2
"from"	4
"front"	6
"frying"	2
"full"	2
"funny"	1
"furiously"	1
"future"	1
"g"	1
"game"	1
"garden"	1
"gardens"	1
"gasped"	1
"gave"	2
"gently"	2
"get"	6
"gets"	1
"getting"	1
"giant"	1
"gingerly"	1
"give"	1
"glance"	1
"glass"	1
"glasses"	3
"glowed"	1
"glumly"	1
"go"	1
"godric"	1
"going"	5
"golden"	1
"gone"	5
"good"	3
"got"	4
"gotten"	2
"great"	2
"green"	1
"greeting"	1
"grew"	2
"grip"	1
"groaned"	1
"growing"	1
"grudgingly"	1
"guess"	1
"had"	33
"hagrid"	14
"hair"	6
"haircut"	1
"haircuts"	1
"half"	3
"hall"	1
"hand"	1
"handkerchief"	2
"hands"	2
"handy"	1
"happen"	1
"hard"	1
"hardly"	1
"harry"	36
"has"	3
"hated"	

In [28]:
%%file nonenglish_mrjob.py

from mrjob.job import MRJob
from spellchecker import SpellChecker
import re

spell = SpellChecker()

class MRNonEnglish(MRJob):

    def mapper(self, _, line):
        words = re.findall(r"[a-zA-Z]+", line.lower())

        for word in words:
            if word not in spell:
                yield word, 1

    def reducer(self, key, values):
        yield key, sum(values)

if __name__ == "__main__":
    MRNonEnglish.run()

Overwriting nonenglish_mrjob.py


In [29]:
# Run the non-English word MapReduce job

!python nonenglish_mrjob.py file2.txt > non_english_output.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/nonenglish_mrjob.root.20260226.054718.116702
Running step 1 of 1...
job output is in /tmp/nonenglish_mrjob.root.20260226.054718.116702/output
Streaming final output from /tmp/nonenglish_mrjob.root.20260226.054718.116702/output...
Removing temp directory /tmp/nonenglish_mrjob.root.20260226.054718.116702...


In [30]:
# Step 11: Display full output

with open("non_english_output.txt") as f:
    print(f.read())

"hermione"	6
"hufflepuff"	6
"hufflepuffs"	1
"importantceremony"	1
"jus"	1
"ll"	7
"malfoy"	4
"slytherin"	2
"thegreat"	1
"ve"	7
"weasleys"	1
"wouldn"	1
"yeh"	1
"anexcuse"	1
"asteep"	1
"brocklehurst"	1
"diagon"	1
"didn"	5
"doesn"	1
"dsomehow"	1
"dursleys"	2
"goyle"	6
"gringotts"	1
"gryffindor"	3
"gryffindors"	1
"hadn"	1
"hagrid"	12
"mcgonagall"	11
"oooooh"	1
"oy"	1
"ravenclaw"	4
"ravenclaws"	1
"scabbers"	6



In [31]:
# Download outputs for submission

files.download("wordcount_output.txt")
files.download("non_english_output.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>